# XGBoost 모델을 사용한 Power 예측
## 1. Ionic5 차량 데이터 로드
## 2. Ionic5 차량의 Power 및 Energy Graph
## 3. Residual(physic-based model - data) 을 XGBoost 모델을 사용한 Power 예측
## 4. Trained 모델을 사용하여 Power 예측 후 Power Graph 및 Energy Graph

## 1. 차량 데이터 로드

In [ ]:
import os
import platform
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from GS_preprocessing import load_data_by_vehicle
from GS_Merge_Power import select_vehicle
from GS_vehicle_dict import vehicle_dict

# Ionic5 차량 데이터 로드
selected_car = 'Ionic5'
if platform.system() == "Windows":
    folder_path = os.path.normpath(r'D:\SamsungSTF\Processed_Data')
elif platform.system() == "Darwin":
    folder_path = os.path.normpath('/Users/wsong/Documents/삼성미래과제/Processed_data')
else:
    raise SystemError("Unknown system.")

folder_path = os.path.join(folder_path, 'TripByTrip')
EV = select_vehicle(2)  # Ionic5의 경우 2번 차량 선택
vehicle_files = load_data_by_vehicle(folder_path, vehicle_dict, selected_car)
vehicle_file_lists = vehicle_files.get(selected_car, [])
vehicle_file_lists.sort()

if not vehicle_files:
    raise FileNotFoundError(f"No files found for the selected vehicle: {selected_car}")

print(f"Data loaded for {selected_car}")


## 2. Ionic5 차량의 Power 및 Energy Graph

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
from scipy.interpolate import griddata
from scipy.stats import linregress

def plot_power(file_lists, folder_path, Target):
    for file in tqdm(file_lists):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        t = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
        t_diff = t.diff().dt.total_seconds().fillna(0)
        t_diff = np.array(t_diff.fillna(0))
        t_min = (t - t.iloc[0]).dt.total_seconds() / 60  # Convert time difference to minutes

        data_power = np.array(data['Power_IV']) / 1000
        model_power = np.array(data['Power']) / 1000
        power_diff = data_power - model_power

        if 'Predicted_Power' in data.columns:
            predicted_power = np.array(data['Predicted_Power']) / 1000

        if Target == 'comparison':
            # Plot the comparison graph
            plt.figure(figsize=(10, 6))  # Set the size of the graph
            plt.xlabel('Time (minutes)')
            plt.ylabel('Data Power and Model Power (kW)')
            plt.plot(t_min, data_power, label='Data Power (kW)', color='tab:blue')
            plt.plot(t_min, model_power, label='model Power (kW)', color='tab:red')
            #plt.plot(t_min, predicted_power, label='Predicted Power (kW)', color='tab:green')
            plt.ylim([-100, 100])

            # Add date and file name
            date = t.iloc[0].strftime('%Y-%m-%d')
            plt.text(1, 1, date, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='right', color='black')
            plt.text(0, 1, 'File: ' + file, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='left', color='black')

            plt.legend(loc='upper left', bbox_to_anchor=(0, 0.97))
            plt.title('Data Power vs.  Model Power')
            plt.tight_layout()
            plt.show()

        else:
            print("Invalid Target")
            return

# Plotting Power Comparison
print("Plotting Power Comparison")
plot_power(vehicle_file_lists[:2], folder_path, 'comparsion')

def plot_energy(file_lists, folder_path, Target):
    for file in tqdm(file_lists):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        t = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
        t_diff = t.diff().dt.total_seconds().fillna(0)
        t_diff = np.array(t_diff.fillna(0))
        t_min = (t - t.iloc[0]).dt.total_seconds() / 60  # Convert time difference to minutes

        data_power = np.array(data['Power_IV'])
        data_energy = data_power * t_diff / 3600 / 1000
        data_energy_cumulative = data_energy.cumsum()
        
        if 'Power' in data.columns:
            model_power = np.array(data['Power'])
            model_energy = model_power * t_diff / 3600 / 1000
            model_energy_cumulative = model_energy.cumsum()

        if 'Predicted_Power' in data.columns:
            predicted_power = np.array(data['Predicted_Power'])
            predicted_energy = predicted_power * t_diff / 3600 / 1000
            predicted_energy_cumulative = predicted_energy.cumsum()

        else:
            pass

        if Target == 'comparison':
            # Plot the comparison graph
            plt.figure(figsize=(10, 6))  # Set the size of the graph
            plt.xlabel('Time (minutes)')
            plt.ylabel('BMS Energy and Model Energy (kWh)')
            plt.plot(t_min, model_energy_cumulative, label='Model Energy (kWh)', color='tab:red')
            plt.plot(t_min, data_energy_cumulative, label='Data Energy (kWh)', color='tab:blue')
            # plt.plot(t_min, predicted_energy_cumulative, label='Trained Model Energy (kWh)', color='tab:green')

            # Add date and file name
            date = t.iloc[0].strftime('%Y-%m-%d')
            plt.text(1, 1, date, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='right', color='black')
            plt.text(0, 1, 'File: ' + file, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='left', color='black')

            plt.legend(loc='upper left', bbox_to_anchor=(0, 0.97))
            plt.title('Model Energy vs. BMS Energy')
            plt.tight_layout()
            plt.show()

        else:
            print("Invalid Target")
            return

# Plotting Energy Comparison
print("Plotting Energy Comparison")
plot_energy(vehicle_file_lists[:2], folder_path, 'comparison')

def plot_energy_scatter(file_lists, folder_path, selected_car, Target):
    data_energies = []
    mod_energies = []
    predicted_energies = []

    # 랜덤으로 1000개의 파일을 선택
    sample_size = min(1000, len(file_lists))
    sampled_files = random.sample(file_lists, sample_size)

    for file in tqdm(sampled_files):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        t = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
        t_diff = t.diff().dt.total_seconds().fillna(0)
        t_diff = np.array(t_diff.fillna(0))

        data_power = np.array(data['Power_IV'])
        data_energy = data_power * t_diff / 3600 / 1000
        data_energies.append(data_energy.cumsum()[-1])

        if 'Power' in data.columns:
            model_power = np.array(data['Power'])
            model_energy = model_power * t_diff / 3600 / 1000
            mod_energies.append(model_energy.cumsum()[-1])

        if 'Predicted_Power' in data.columns:
            predicted_power = np.array(data['Predicted_Power'])
            predicted_energy = predicted_power * t_diff / 3600 / 1000
            predicted_energies.append(predicted_energy.cumsum()[-1])

    if Target == 'model':
        fig, ax = plt.subplots(figsize=(6, 6))
        colors = cm.rainbow(np.linspace(0, 1, len(data_energies)))

        ax.set_xlabel('Data Energy (kWh)')
        ax.set_ylabel('Model Energy (kWh)')

        for i in range(len(mod_energies)):
            ax.scatter(data_energies[i], mod_energies[i], color=colors[i], facecolors='none',
                       edgecolors=colors[i], label='Model Energy' if i == 0 else "")

        slope_original, intercept_original, _, _, _ = linregress(data_energies, mod_energies)
        ax.plot(np.array(data_energies), intercept_original + slope_original * np.array(data_energies),
                color='lightblue', label='Trendline')

        lims = [
            np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()]),
        ]
        ax.plot(lims, lims, 'r-', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(0, None)
        ax.set_ylim(0, None)

        plt.legend()
        plt.title(f"{selected_car} : All trip's BMS Energy vs. Model Energy over Time")
        plt.show()

    elif Target == 'fitting':
        fig, ax = plt.subplots(figsize=(6, 6))
        colors = cm.rainbow(np.linspace(0, 1, len(data_energies)))

        ax.set_xlabel('Data Energy (kWh)')
        ax.set_ylabel('Predicted Energy (kWh)')

        for i in range(len(data_energies)):
            ax.scatter(data_energies[i], mod_energies[i], color=colors[i], facecolors='none',
                       edgecolors=colors[i], label='Before fitting' if i == 0 else "")

        for i in range(len(data_energies)):
            ax.scatter(data_energies[i], predicted_energies[i], color=colors[i],
                       label='After fitting' if i == 0 else "")

        slope_original, intercept_original, _, _, _ = linregress(data_energies, mod_energies)
        ax.plot(np.array(data_energies), intercept_original + slope_original * np.array(data_energies),
                color='lightblue', label='Trend (before fitting)')

        slope, intercept, _, _, _ = linregress(data_energies, predicted_energies)
        ax.plot(np.array(data_energies), intercept + slope * np.array(data_energies), 'b',
                label='Trend (after fitting)')

        lims = [
            np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()]),
        ]
        ax.plot(lims, lims, 'r-', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(0, None)
        ax.set_ylim(0, None)

        plt.legend()
        plt.title(f"{selected_car} : All trip's BMS Energy vs. Trained Model Energy over Time")
        plt.show()

# Energy Scatter Plot
print("Plotting Energy Scatter")
plot_energy_scatter(vehicle_file_lists, folder_path, selected_car, 'model')


## 3. Residual(physic-based model - data) 을 XGBoost 모델을 사용한 Power 예측

In [8]:
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def process_files(files):
    # Speed in m/s (160 km/h = 160 / 3.6 m/s)
    SPEED_MIN = 0 / 3.6
    SPEED_MAX = 180 / 3.6
    ACCELERATION_MIN = -15
    ACCELERATION_MAX = 9

    df_list = []
    for file in files:
        try:
            data = pd.read_csv(file)
            if 'Power' in data.columns and 'Power_IV' in data.columns:
                data['Residual'] = data['Power'] - data['Power_IV']
                data['speed'] = data['speed']
                df_list.append(data[['speed', 'acceleration', 'Residual']])
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    if not df_list:
        raise ValueError("No valid data files found. Please check the input files and try again.")

    full_data = pd.concat(df_list, ignore_index=True)

    # Define scaler with the predefined range
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(pd.DataFrame([[SPEED_MIN, ACCELERATION_MIN], [SPEED_MAX, ACCELERATION_MAX]], columns=['speed', 'acceleration']))

    full_data[['speed', 'acceleration']] = scaler.transform(full_data[['speed', 'acceleration']])

    return full_data, scaler

def cross_validate(vehicle_files, selected_vehicle, save_dir="models"):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    results = []
    best_rmse = float('inf')
    best_model = None

    if selected_vehicle not in vehicle_files or not vehicle_files[selected_vehicle]:
        print(f"No files found for the selected vehicle: {selected_vehicle}")
        return

    files = vehicle_files[selected_vehicle]
    data, scaler = process_files(files)
    X = data[['speed', 'acceleration']].to_numpy()
    y = data['Residual'].to_numpy()

    y_mean = np.mean(y)
    y_range = np.ptp(y)  # range of the residuals

    for fold_num, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        params = {
            'tree_method': 'hist',
            'device': 'cuda',
            'eval_metric': ['rmse', 'mae'],
            'objective': 'reg:squarederror'
        }
        evals = [(dtrain, 'train'), (dtest, 'test')]
        model = xgb.train(params, dtrain, num_boost_round=100, evals=evals)
        y_pred = model.predict(dtest)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        nrmse = rmse / y_range
        percent_rmse = (rmse / y_mean) * 100
        results.append((fold_num, rmse, nrmse, percent_rmse))
        print(f"Vehicle: {selected_vehicle}, Fold: {fold_num}, RMSE: {rmse}, NRMSE: {nrmse}, Percent RMSE: {percent_rmse}%")

        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model

    # Save the best model
    if best_model:
        model_file = os.path.join(save_dir, f"XGB_best_model_{selected_vehicle}.json")
        surface_plot = os.path.join(save_dir, f"XGB_best_model_{selected_vehicle}_plot.html")
        best_model.save_model(model_file)
        print(f"Best model for {selected_vehicle} saved with RMSE: {best_rmse}")
        plot_3d(X_test, y_test, y_pred, fold_num, selected_vehicle, scaler, 400, 30)
    return results, scaler

def plot_3d(X, y_true, y_pred, fold_num, vehicle, scaler, num_grids=400, samples_per_grid=30, output_file=None):
    if X.shape[1] != 2:
        print("Error: X should have 2 columns.")
        return

    # 역변환하여 원래 범위로 변환
    X_orig = scaler.inverse_transform(X)

    # Speed를 km/h로 변환
    X_orig[:, 0] *= 3.6

    # 그리드 크기 계산
    num_grid_sqrt = int(np.sqrt(num_grids))
    grid_size_x = (X_orig[:, 0].max() - X_orig[:, 0].min()) / num_grid_sqrt
    grid_size_y = (X_orig[:, 1].max() - X_orig[:, 1].min()) / num_grid_sqrt

    samples = []

    for i in range(num_grid_sqrt):
        for j in range(num_grid_sqrt):
            x_min = X_orig[:, 0].min() + i * grid_size_x
            x_max = x_min + grid_size_x
            y_min = X_orig[:, 1].min() + j * grid_size_y
            y_max = y_min + grid_size_y

            # 해당 그리드 내의 데이터 인덱스 선택
            grid_indices = np.where(
                (X_orig[:, 0] >= x_min) & (X_orig[:, 0] < x_max) &
                (X_orig[:, 1] >= y_min) & (X_orig[:, 1] < y_max)
            )[0]

            if len(grid_indices) > 0:
                sample_size = min(samples_per_grid, len(grid_indices))
                sample_indices = np.random.choice(grid_indices, sample_size, replace=False)
                samples.append(sample_indices)

    samples = np.concatenate(samples)
    X_sampled = X_orig[samples]
    y_true_sampled = y_true[samples]
    y_pred_sampled = y_pred[samples]

    trace1 = go.Scatter3d(
        x=X_sampled[:, 0], y=X_sampled[:, 1], z=y_true_sampled,
        mode='markers',
        marker=dict(size=5, color='blue', opacity=0.8),
        name='Actual Residual'
    )
    trace2 = go.Scatter3d(
        x=X_sampled[:, 0], y=X_sampled[:, 1], z=y_pred_sampled,
        mode='markers',
        marker=dict(size=5, color='red', opacity=0.8),
        name='Predicted Residual'
    )

    grid_x, grid_y = np.linspace(X_orig[:, 0].min(), X_orig[:, 0].max(), 100), np.linspace(X_orig[:, 1].min(), X_orig[:, 1].max(), 100)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)
    grid_z = griddata((X_orig[:, 0], X_orig[:, 1]), y_pred, (grid_x, grid_y), method='linear')

    surface_trace = go.Surface(
        x=grid_x,
        y=grid_y,
        z=grid_z,
        colorscale='Viridis',
        name='Predicted Residual Surface',
        opacity=0.7
    )

    data = [trace1, trace2, surface_trace]
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene=dict(
            xaxis=dict(title='Speed (km/h)'),
            yaxis=dict(title='Acceleration (m/s²)'),
            zaxis=dict(title='Residual'),
        ),
        title=f'3D Plot of Actual vs. Predicted Residuals (Fold {fold_num}, Vehicle: {vehicle})'
    )
    fig = go.Figure(data=data, layout=layout)
    if output_file:
        fig.write_html(output_file)
    else:
        fig.show()

save_dir = os.path.join(os.path.dirname(folder_path), 'Models')

results, scaler = cross_validate(vehicle_files, selected_car, save_dir=save_dir)

# Save the scaler
scaler_path = os.path.join(save_dir, f'XGB_scaler_{selected_car}.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"Scaler saved at {scaler_path}")

# Print overall results
if results:
    for fold_num, rmse, nrmse, percent_rmse in results:
        print(f"Fold: {fold_num}, RMSE: {rmse}, NRMSE: {nrmse}, Percent RMSE: {percent_rmse}")
else:
    print(f"No results for the selected vehicle: {selected_car}")

C:\Users\WSONG\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:17:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


[0]	train-rmse:10447.44568	train-mae:6558.23953	test-rmse:10413.86959	test-mae:6557.32365
[1]	train-rmse:10306.12809	train-mae:6402.90038	test-rmse:10275.67200	test-mae:6402.19574
[2]	train-rmse:10231.84370	train-mae:6320.42469	test-rmse:10204.24475	test-mae:6319.98020
[3]	train-rmse:10192.58104	train-mae:6277.36034	test-rmse:10166.68131	test-mae:6276.98997
[4]	train-rmse:10171.39840	train-mae:6255.23844	test-rmse:10146.86868	test-mae:6255.01344
[5]	train-rmse:10160.62899	train-mae:6244.10075	test-rmse:10136.57820	test-mae:6243.85619
[6]	train-rmse:10154.56078	train-mae:6239.46575	test-rmse:10131.34724	test-mae:6239.21559
[7]	train-rmse:10150.60697	train-mae:6236.62778	test-rmse:10128.57556	test-mae:6236.36353
[8]	train-rmse:10147.46594	train-mae:6233.86096	test-rmse:10126.09812	test-mae:6233.61421
[9]	train-rmse:10145.69684	train-mae:6232.70595	test-rmse:10124.82943	test-mae:6232.46378
[10]	train-rmse:10144.66225	train-mae:6231.98061	test-rmse:10124.27295	test-mae:6231.73994
[11]	trai

C:\Users\WSONG\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:20:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


[0]	train-rmse:10436.23714	train-mae:6557.80716	test-rmse:10464.43293	test-mae:6556.71767
[1]	train-rmse:10296.20750	train-mae:6403.23413	test-rmse:10321.99688	test-mae:6401.64485
[2]	train-rmse:10222.34511	train-mae:6320.91020	test-rmse:10247.09719	test-mae:6319.13522
[3]	train-rmse:10183.49812	train-mae:6276.69968	test-rmse:10207.53689	test-mae:6274.82870
[4]	train-rmse:10163.30264	train-mae:6254.80602	test-rmse:10186.96658	test-mae:6252.92770
[5]	train-rmse:10152.18700	train-mae:6245.02085	test-rmse:10175.41920	test-mae:6243.10821
[6]	train-rmse:10146.06878	train-mae:6239.21381	test-rmse:10169.28873	test-mae:6237.30882
[7]	train-rmse:10142.11829	train-mae:6236.01450	test-rmse:10165.01467	test-mae:6234.14711
[8]	train-rmse:10138.97188	train-mae:6233.35242	test-rmse:10161.79307	test-mae:6231.52140
[9]	train-rmse:10137.65373	train-mae:6232.55450	test-rmse:10160.17807	test-mae:6230.74141
[10]	train-rmse:10136.61333	train-mae:6231.88302	test-rmse:10159.08477	test-mae:6230.13053
[11]	trai

C:\Users\WSONG\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:24:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


[0]	train-rmse:10441.83827	train-mae:6557.87844	test-rmse:10438.47274	test-mae:6556.88920
[1]	train-rmse:10300.63545	train-mae:6402.37694	test-rmse:10298.88323	test-mae:6401.90365
[2]	train-rmse:10226.29198	train-mae:6320.23222	test-rmse:10225.80289	test-mae:6320.32749
[3]	train-rmse:10187.19254	train-mae:6276.98617	test-rmse:10187.58044	test-mae:6277.36451
[4]	train-rmse:10166.50688	train-mae:6254.60970	test-rmse:10167.49383	test-mae:6255.19019
[5]	train-rmse:10155.53708	train-mae:6243.48189	test-rmse:10156.75991	test-mae:6244.21191


KeyboardInterrupt: 

In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed

def process_file_with_trained_model(file, model, scaler):
    try:
        data = pd.read_csv(file)
        if 'speed' in data.columns and 'acceleration' in data.columns and 'Power_IV' in data.columns:
            # Use the provided scaler
            features = data[['speed', 'acceleration']]
            features_scaled = scaler.transform(features)

            predicted_residual = model.predict(features_scaled)

            data['Predicted_Power'] = data['Power'] - predicted_residual

            # Save the updated file
            data.to_csv(file, index=False)

            print(f"Processed file {file}")
        else:
            print(f"File {file} does not contain required columns 'speed', 'acceleration', or 'Power_IV'.")
    except Exception as e:
        print(f"Error processing file {file}: {e}")

def add_predicted_power_column(files, model_path, scaler):
    try:
        model = xgb.XGBRegressor()
        model.load_model(model_path)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_file_with_trained_model, file, model, scaler) for file in files]
        for future in as_completed(futures):
            future.result()


model_path = os.path.join(os.path.dirname(folder_path), 'Models',
                          f'XGB_best_model_{selected_car}.json')
scaler_path = os.path.join(os.path.dirname(folder_path), 'Models',
                           f'XGB_scaler_{selected_car}.pkl')

if not vehicle_file_lists:
    print(f"No files to process for the selected vehicle: {selected_car}")
    return

# Load the scaler
with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)

add_predicted_power_column(vehicle_file_lists, model_path, scaler)

## 4. Trained 모델을 사용하여 Power 예측 후 Power Graph 및 Energy Graph

In [ ]:
def plot_power_fitting(file_lists, folder_path, Target):
    for file in tqdm(file_lists):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        t = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
        t_diff = t.diff().dt.total_seconds().fillna(0)
        t_diff = np.array(t_diff.fillna(0))
        t_min = (t - t.iloc[0]).dt.total_seconds() / 60  # Convert time difference to minutes

        data_power = np.array(data['Power_IV']) / 1000
        model_power = np.array(data['Power']) / 1000
        power_diff = data_power - model_power

        if 'Predicted_Power' in data.columns:
            predicted_power = np.array(data['Predicted_Power']) / 1000

        if Target == 'comparison':
            # Plot the comparison graph
            plt.figure(figsize=(10, 6))  # Set the size of the graph
            plt.xlabel('Time (minutes)')
            plt.ylabel('Data Power and Model Power (kW)')
            plt.plot(t_min, data_power, label='Data Power (kW)', color='tab:blue')
            plt.plot(t_min, model_power, label='model Power (kW)', color='tab:red')
            plt.plot(t_min, predicted_power, label='Predicted Power (kW)', color='tab:green')
            plt.ylim([-100, 100])

            # Add date and file name
            date = t.iloc[0].strftime('%Y-%m-%d')
            plt.text(1, 1, date, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='right', color='black')
            plt.text(0, 1, 'File: ' + file, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='left', color='black')

            plt.legend(loc='upper left', bbox_to_anchor=(0, 0.97))
            plt.title('Data Power vs.  Model Power')
            plt.tight_layout()
            plt.show()

        else:
            print("Invalid Target")
            return

# Plotting Power Comparison
print("Plotting Power Comparison")
plot_power_fitting(vehicle_file_lists[:2], folder_path, 'comparsion')

def plot_energy_fitting(file_lists, folder_path, Target):
    for file in tqdm(file_lists):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        t = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
        t_diff = t.diff().dt.total_seconds().fillna(0)
        t_diff = np.array(t_diff.fillna(0))
        t_min = (t - t.iloc[0]).dt.total_seconds() / 60  # Convert time difference to minutes

        data_power = np.array(data['Power_IV'])
        data_energy = data_power * t_diff / 3600 / 1000
        data_energy_cumulative = data_energy.cumsum()
        
        if 'Power' in data.columns:
            model_power = np.array(data['Power'])
            model_energy = model_power * t_diff / 3600 / 1000
            model_energy_cumulative = model_energy.cumsum()

        if 'Predicted_Power' in data.columns:
            predicted_power = np.array(data['Predicted_Power'])
            predicted_energy = predicted_power * t_diff / 3600 / 1000
            predicted_energy_cumulative = predicted_energy.cumsum()

        else:
            pass

        if Target == 'comparison':
            # Plot the comparison graph
            plt.figure(figsize=(10, 6))  # Set the size of the graph
            plt.xlabel('Time (minutes)')
            plt.ylabel('BMS Energy and Model Energy (kWh)')
            plt.plot(t_min, model_energy_cumulative, label='Model Energy (kWh)', color='tab:red')
            plt.plot(t_min, data_energy_cumulative, label='Data Energy (kWh)', color='tab:blue')
            plt.plot(t_min, predicted_energy_cumulative, label='Trained Model Energy (kWh)', color='tab:green')

            # Add date and file name
            date = t.iloc[0].strftime('%Y-%m-%d')
            plt.text(1, 1, date, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='right', color='black')
            plt.text(0, 1, 'File: ' + file, transform=plt.gca().transAxes, fontsize=12,
                     verticalalignment='top', horizontalalignment='left', color='black')

            plt.legend(loc='upper left', bbox_to_anchor=(0, 0.97))
            plt.title('Model Energy vs. BMS Energy')
            plt.tight_layout()
            plt.show()

        else:
            print("Invalid Target")
            return

# Plotting Energy Comparison
print("Plotting Energy Comparison")
plot_energy_fitting(vehicle_file_lists[:2], folder_path, 'comparison')

def plot_energy_scatter(file_lists, folder_path, selected_car, Target):
    data_energies = []
    mod_energies = []
    predicted_energies = []

    # 랜덤으로 1000개의 파일을 선택
    sample_size = min(1000, len(file_lists))
    sampled_files = random.sample(file_lists, sample_size)

    for file in tqdm(sampled_files):
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        t = pd.to_datetime(data['time'], format='%Y-%m-%d %H:%M:%S')
        t_diff = t.diff().dt.total_seconds().fillna(0)
        t_diff = np.array(t_diff.fillna(0))

        data_power = np.array(data['Power_IV'])
        data_energy = data_power * t_diff / 3600 / 1000
        data_energies.append(data_energy.cumsum()[-1])

        if 'Power' in data.columns:
            model_power = np.array(data['Power'])
            model_energy = model_power * t_diff / 3600 / 1000
            mod_energies.append(model_energy.cumsum()[-1])

        if 'Predicted_Power' in data.columns:
            predicted_power = np.array(data['Predicted_Power'])
            predicted_energy = predicted_power * t_diff / 3600 / 1000
            predicted_energies.append(predicted_energy.cumsum()[-1])

    if Target == 'model':
        fig, ax = plt.subplots(figsize=(6, 6))
        colors = cm.rainbow(np.linspace(0, 1, len(data_energies)))

        ax.set_xlabel('Data Energy (kWh)')
        ax.set_ylabel('Model Energy (kWh)')

        for i in range(len(mod_energies)):
            ax.scatter(data_energies[i], mod_energies[i], color=colors[i], facecolors='none',
                       edgecolors=colors[i], label='Model Energy' if i == 0 else "")

        slope_original, intercept_original, _, _, _ = linregress(data_energies, mod_energies)
        ax.plot(np.array(data_energies), intercept_original + slope_original * np.array(data_energies),
                color='lightblue', label='Trendline')

        lims = [
            np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()]),
        ]
        ax.plot(lims, lims, 'r-', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(0, None)
        ax.set_ylim(0, None)

        plt.legend()
        plt.title(f"{selected_car} : All trip's BMS Energy vs. Model Energy over Time")
        plt.show()

    elif Target == 'fitting':
        fig, ax = plt.subplots(figsize=(6, 6))
        colors = cm.rainbow(np.linspace(0, 1, len(data_energies)))

        ax.set_xlabel('Data Energy (kWh)')
        ax.set_ylabel('Predicted Energy (kWh)')

        for i in range(len(data_energies)):
            ax.scatter(data_energies[i], mod_energies[i], color=colors[i], facecolors='none',
                       edgecolors=colors[i], label='Before fitting' if i == 0 else "")

        for i in range(len(data_energies)):
            ax.scatter(data_energies[i], predicted_energies[i], color=colors[i],
                       label='After fitting' if i == 0 else "")

        slope_original, intercept_original, _, _, _ = linregress(data_energies, mod_energies)
        ax.plot(np.array(data_energies), intercept_original + slope_original * np.array(data_energies),
                color='lightblue', label='Trend (before fitting)')

        slope, intercept, _, _, _ = linregress(data_energies, predicted_energies)
        ax.plot(np.array(data_energies), intercept + slope * np.array(data_energies), 'b',
                label='Trend (after fitting)')

        lims = [
            np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()]),
        ]
        ax.plot(lims, lims, 'r-', alpha=0.75, zorder=0)
        ax.set_aspect('equal')
        ax.set_xlim(0, None)
        ax.set_ylim(0, None)

        plt.legend()
        plt.title(f"{selected_car} : All trip's BMS Energy vs. Trained Model Energy over Time")
        plt.show()

# Energy Scatter Plot
print("Plotting Energy Scatter")
plot_energy_scatter(vehicle_file_lists, folder_path, selected_car, 'fitting')